In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc, sys, warnings, random, math, psutil, pickle

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [ ]:
weather_train = pd.read_csv(
    '../../data-vol-1/weather_train.csv'
)

weather_train.timestamp = pd.to_datetime(weather_train.timestamp, format='%Y-%m-%d %H:%M:%S')

weather_test = pd.read_csv(
    "../../data-vol-1/weather_test.csv"
)

weather_test.timestamp = pd.to_datetime(weather_test.timestamp, format='%Y-%m-%d %H:%M:%S')

In [ ]:
external_weather = pd.read_csv(
    '../../data-vol-1/more_weather_locations.csv',
    index_col=0,
    parse_dates=True,
    infer_datetime_format=True,
).join(
    pd.read_csv(
        '../../data-vol-1/temperature.csv',
        index_col=0,
        parse_dates=True,
        infer_datetime_format=True,
    ).sub(273.15),
    how='left',
)

In [ ]:
external_weather.head()

## [Time Zone](https://www.kaggle.com/datadugong/locate-better-cities-by-weather-temp-fill-nans)

if there is time zone details, then no need to align the timestamp of each site

### Holidays in the city/country


### City

In [ ]:
weather = pd.concat([weather_train, weather_test]).set_index(['site_id', 'timestamp'])

weather = weather.reindex(
    pd.MultiIndex.from_product(
        [range(16), pd.date_range('2016-01-01', '2018-12-31 23:00', freq='H')])
)

weather = weather.unstack(level=0).interpolate(limit=2).ffill(limit=1).bfill(limit=1)

In [ ]:
w = weather_test.set_index(['site_id', 'timestamp'])
w = w.unstack('site_id')

to_lieaner_interp = []
to_lieaner_interp_idx = []

for col, S in w.iteritems():
    to_lieaner_interp.append(
        S.isnull().astype(int).groupby(S.notnull().astype(int).cumsum()).sum().max()
    )

    to_lieaner_interp_idx.append(col)

pd.DataFrame(to_lieaner_interp, index=pd.MultiIndex.from_tuples(to_lieaner_interp_idx)).unstack(0).rename(columns={0: 'max_consecutive_nans'}, level=0)

### Timezone, Latitude, Longtitude

In [2]:
# in site_id.csv

site_info = weather_train = pd.read_csv('../../data-vol-1/cities_info.csv')
site_info

,site_id,location,state,lat_long,low_elevation,high_elevation,timezone,utc,country
0,0,orlando,florida,"(28.512274, -81.40619)",25.00,25.00,US/Eastern,-04:00,US
1,1,london,NaN,"(51.471092, -0.455046)",19.00,19.00,Europe/London,+01:00,UK
2,2,phoenix,arizona,"(33.47425, -112.077456)",347.47,455.68,US/Mountain,-07:00,US
3,3,washington,washingtondc,"(38.8973, -77.02894)",0.00,125.00,US/Eastern,-04:00,US
4,4,berkeley,california,"(37.8719, -122.2585)",52.00,52.00,US/Pacific,-07:00,US
5,5,birmingham,NaN,"(52.452442, -1.743035)",19.00,19.00,Europe/London,+01:00,UK
6,6,washington,washingtondc,"(38.8973, -77.02894)",12.00,12.00,US/Eastern,-04:00,US
7,7,ottowa,NaN,"(45.414524, -75.711136)",6.00,233.00,Canada/Eastern,-04:00,CA
8,8,orlando,florida,"(28.512274, -81.40619)",25.00,25.00,US/Eastern,-04:00,US
9,9,austin,texas,"(30.2849,-97.7341)",88.00,405.00,US/Central,-05:00,US


### sun info

In [ ]:
!pip install pysolar

In [3]:
import math
from datetime import timedelta
from pyephem_sunpath.sunpath import sunpos
from datetime import datetime, timedelta
from pysolar.solar import *

df = pd.DataFrame(columns=['site_id','local_time','utc_time','altitude','azimuth','radiation'])

for index, row in site_info.iterrows():
    tz = int(str(row['utc'][:3]))
    site_id = row['site_id']
    lat_long = row['lat_long'].replace('(','').replace(')','').split(',')
    lat = float(lat_long[0])
    long = float(lat_long[1])
    for i in pd.date_range(start='2015-12-31 16:00:00', end='2019-01-01 09:00:00', freq='H'):
        # all output fits in local time      
        u = i - timedelta(hours=tz)
        d = datetime.datetime(i.year, i.month, i.day, i.hour, tzinfo=datetime.timezone.utc)
        alt, azm = sunpos(i, lat, long, tz, dst=False)
        r = radiation.get_radiation_direct(d, alt)
        df = df.append({'site_id':site_id,'local_time':i,'utc_time':u,'altitude':alt,'azimuth':azm,'radiation':r}, ignore_index=True)
    
#to_csv(site_id + '_' + site +'_sun.csv')

In [4]:
df.head()

,site_id,local_time,utc_time,altitude,azimuth,radiation
0,0,2015-12-31 16:00:00,2015-12-31 20:00:00,26.858237,219.238171,907.112304
1,0,2015-12-31 17:00:00,2015-12-31 21:00:00,17.583163,230.260567,778.502534
2,0,2015-12-31 18:00:00,2015-12-31 22:00:00,6.895952,239.099760,386.766774
3,0,2015-12-31 19:00:00,2015-12-31 23:00:00,-4.214855,246.389157,0.000000
4,0,2015-12-31 20:00:00,2016-01-01 00:00:00,-17.282735,252.673395,0.000000


In [6]:
df = pd.read_csv(r'../../data-vol-1/dates_related/radiance.csv')

In [7]:
df.head()

,Unnamed: 0,site_id,local_time,utc_time,altitude,azimuth,radiation
0,0,0,2015-12-31 16:00:00,2015-12-31 20:00:00,26.858237,219.238171,907.112304
1,1,0,2015-12-31 17:00:00,2015-12-31 21:00:00,17.583163,230.260567,778.502534
2,2,0,2015-12-31 18:00:00,2015-12-31 22:00:00,6.895952,239.099760,386.766774
3,3,0,2015-12-31 19:00:00,2015-12-31 23:00:00,-4.214855,246.389157,0.000000
4,4,0,2015-12-31 20:00:00,2016-01-01 00:00:00,-17.282735,252.673395,0.000000


In [5]:
df.to_csv(r'../../data-vol-1/dates_related/radiance.csv')

Below are backup code

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(weather_train)

In [ ]:
msno.matrix(weather_test)

The missingno correlation heatmap measures nullity correlation: how strongly the presence or absence of one variable affects the presence of another:

In [ ]:
a = msno.heatmap(weather_train, sort='ascending')

## Fill NA in weather and missing timestamp

### Wind_Speed
### Wind_direction
### Sea_level_pressure
- no values for site 5
### Precip_depth_1_hr
- no values for site 1, 5, 12
### Dew_temperature
### Cloud_coverage:
- much higher on site 12 than on all other sites
- no values for site 7, 11
### Air_temperature
### Dew temperture

In [ ]:
# NOAA Calculation, results is wrong

# fractional year: r
    r = (2*math.pi/365) * (i.timetuple().tm_yday - 1 + (i.hour-12)/24)
    eqtime = 229.18*(0.000075 + 0.001868*math.cos(r) - 0.032077*math.sin(r) - 0.014615*math.cos(2*r) 
                     - 0.040849*math.sin(2*r))
    print('eqtime:' + str(eqtime))
    decl = 0.006918 - 0.399912*math.cos(r) + 0.070257*math.sin(r) - 0.006758*math.cos(2*r) + 0.000907*math.sin(2*r) - 0.002697*math.cos(3*r) + 0.00148*math.sin(3*r)
        
    if ((i >= dateutil.parser.parse(DST_Start)) or i < dateutil.parser.parse(DST_End)):
        utc = summer_UTC
    
    if ((i < dateutil.parser.parse(DST_Start)) or i >= dateutil.parser.parse(DST_End)):
        utc = winter_UTC
        
    time_offset = eqtime + 4*long - 60*utc
    print('time_offset: ' + str(time_offset))
    
    # true solar time
    tst = i.hour*60 + i.minute + i.second/60 + time_offset
    
    # solar hour angle
    ha = (tst/4) - 180
    
    # solar zenith angle
    zenith = math.acos(math.sin(lat*math.pi/180)*math.sin(decl) + math.cos(lat*math.pi/180)*math.cos(decl)*math.cos(ha))
    
    # solar azimuth
    azimuth = 180- math.acos((math.sin(decl)-math.sin(lat*math.pi/180)*math.cos(zenith))/(math.cos(lat*math.pi/180)*math.sin(zenith)))
    
    # sunrise
    sunrise_angle = np.arccos(math.cos(90.833*math.pi/180)/(math.cos(lat*math.pi/180)*math.cos(decl))-math.tan(lat*math.pi/180)*math.tan(decl))
    print('sunrise_angle:' + str(sunrise_angle))
    # sunset
    sunset_angle = -np.arccos(math.cos(90.833*math.pi/180)/(math.cos(lat*math.pi/180)*math.cos(decl))-math.tan(lat*math.pi/180)*math.tan(decl))
    print('sunset_angle:' + str(sunset_angle))
    
    sunrise_utc = 720 - 4*(long + sunrise_angle) - eqtime
    sunset_utc = 720 - 4*(long + sunset_angle) - eqtime
    
    print('sunrise_utc: ' + str(sunrise_utc/60))
    print('sunset_utc: ' + str(sunset_utc/60))
    
    snoon = 720 - 4*long - eqtime
    df = df.append({'datetime':i,'utc':utc,'utc_time':i + timedelta(hours=utc),'r':r,'eqtime':eqtime,'decl':decl,'tst':tst,
                    'ha':ha,'zenith':zenith,'azimuth':azimuth,'sunset_angle':sunset_angle, 
                    'sunrise_angle':sunrise_angle,'sunrise_utc':sunrise_utc/60,
                    'sunset_utc':sunset_utc/60,'snoon':snoon/60},ignore_index = True)
   